In [ ]:
using Revise
import TensorCrossInterpolation as TCI
using MSSTA
using ITensors
ITensors.disable_warn_order()
using SparseIR: valueim, FermionicFreq
using BasicInterpolators
using Plots
using FFTW

In [ ]:
ek(kx, ky) = 2*cos(kx) + 2*cos(ky) - 1.0

function gk(kx, ky, β)
    iv = valueim(FermionicFreq(1), β)
    1 / (iv - ek(kx, ky))
end

function f(xs, β)::ComplexF64
    @assert all(0 .<= xs .<= 1.)
    kxy = 2π .* xs
    return gk(kxy[1], kxy[2], β)
end

In [ ]:
R = 12
N = 2^R
β = 100.0

halfN = 2^(R-1)
siteskx = [Index(2, "Qubit, kx=$n") for n in 1:R]
sitesky = [Index(2, "Qubit, ky=$n") for n in 1:R]
sitesx = [Index(2, "Qubit, x=$n") for n in 1:R]
sitesy = [Index(2, "Qubit, y=$n") for n in 1:R]
#sitesk = [Index(4, "Quantics, k=$n") for n in 1:R]

In [ ]:

xs = collect(LinRange(0, 1, 2^R))
ys = collect(LinRange(0, 1, 2^R))
xys = [(x, y) for x in xs, y in ys]

zs = f.(xys, β)
heatmap(abs.(zs))

In [ ]:
zs_r = fft(zs)
heatmap(log.(abs.(zs_r)))

In [ ]:
#zs_reconst = ifft(zs_r)
#heatmap(abs.(zs_reconst))

In [ ]:
aqtt = MSSTA.construct_adaptiveqtt(ComplexF64, Val(2),
    x->f(x, β), R; maxiter=100, tolerance=1e-5)

In [ ]:
intpl_re = BilinearInterpolator(xs, ys, real.(zs_r))
intpl_im = BilinearInterpolator(xs, ys, imag.(zs_r))

aqtt = MSSTA.construct_adaptiveqtt(ComplexF64, Val(2),
    x->(intpl_re(x...) + im * intpl_im(x...)), R; maxiter=100, tolerance=1e-5)

In [ ]:
#M = MSSTA.asmps(aqtt, sitesk)
#truncate!(M; cutoff=1e-8)
#semilogy(dim.(linkinds(M)), marker="x")
#semilogy(MSSTA.maxlinkdims(siteinds(M)))